# Sentiment Analysis on Movie review

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from sklearn.utils import shuffle
# LIbrary to Clean the texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
ps = PorterStemmer()

## Reading Train Data

In [3]:
df=[]
for filename in glob.glob(os.path.join('train/pos/', '*.txt')):
    f = open(filename, 'r')
    content = f.read()
    df.append([content,1]) #1 stands for positive review
    
for filename in glob.glob(os.path.join('train/neg/', '*.txt')):
    f = open(filename, 'r')
    content = f.read()
    df.append([content,0]) #0 stands for negative review
    
train=pd.DataFrame(df)
train=train.sample(frac=1).reset_index(drop=True)
train.head()

,0,1
0,Everyone involved with this project should be ...,0
1,What the movie The 60s really represents (to t...,1
2,If you would like to see a film of different k...,1
3,Back when musicals weren't showcases for chore...,1
4,I figured that it's about time I let this one ...,0


## Reading Testing Data

In [4]:
df=[]
for filename in glob.glob(os.path.join('test/pos/', '*.txt')):
    f = open(filename, 'r')
    content = f.read()
    df.append([content,1]) #1 stands for positive review
    
for filename in glob.glob(os.path.join('test/neg/', '*.txt')):
    f = open(filename, 'r')
    content = f.read()
    df.append([content,0]) #0 stands for negative review
    
test=pd.DataFrame(df)
test=test.sample(frac=1).reset_index(drop=True)
test.head()

,0,1
0,A suprisingly good film considering the circum...,1
1,This movie was so poorly acted. What was with ...,0
2,"To my eternal shame, I've never seen a silent ...",1
3,My yardstick for measuring a movie's watch-abi...,1
4,This movie was the worst movie I've ever seen....,0


## Cleaning the reviews

In [5]:
 def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def getCleanReview(review):
    review = remove_html_tags(review)
    #only keep alphabets remove rest
    review = re.sub('[^a-zA-Z]', ' ', review)
    #turn all reviews into lowercase
    review = review.lower()
    #remove stopwords and do stemming
    #review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    #Lemmatize and merge the review together after making all the changes
    #review = ' '.join([lemmatizer.lemmatize(word) for word in review])
    return review

In [7]:
X_train=train.iloc[:,0].values
Y_train=train.iloc[:,1].values
for i in range(len(X_train)):
    X_train[i]=getCleanReview(X_train[i])
    
#clean X_test as well 
X_test=test.iloc[:,0].values
Y_test=test.iloc[:,1].values

for i in range(len(X_test)):
    X_test[i]=getCleanReview(X_test[i])

# Naive bayes from scratch

In [10]:
wordfreq={}
for review in X_train:
    words=review.split()
    for word in words:
        if word not in wordfreq.keys():
            wordfreq[word]=1
        else:
            wordfreq[word]+=1
len(wordfreq)

74218

In [12]:
#remove words with occurance less than 5 and remove stopwords
repeat=[]
for i in wordfreq.keys():
    if wordfreq[i] <5 :
        repeat.append(i)
    
for i in repeat:
    del wordfreq[i]

len(wordfreq)

28757

## Drop the less frequent words from out training data

In [13]:
def remove_extra_words(dataset,wordfreq):
    for index ,review in enumerate(dataset):
        cleanreview=[]
        for word in review.split():
            if word in wordfreq.keys():
                cleanreview.append(word)
        cleanreview=' '.join(cleanreview)
        dataset[index]=cleanreview
    return dataset

X_train=remove_extra_words(X_train,wordfreq)

### Functions required for naive bayes

In [15]:
def Word_count(X,wordfreq):
    wordcount={}
    for word in wordfreq.keys():
        count=0
        for review in X:
            if word in review:
                count+=1
        wordcount[word]=count
    return wordcount

wordcount=Word_count(X_train,wordfreq)

#occurance of the word in positive document
def Word_positive(X,Y,wordfreq):
    wordpositive={}
    for word in wordfreq.keys():
        count=0
        for index,review in enumerate(X):
            if word in review and Y[index]==1:
                count+=1
        wordpositive[word]=count
    return wordpositive

wordpositive=Word_positive(X_train,Y_train,wordfreq)

#occurance of the word in negative document
def Word_negative(X,Y,wordfreq):
    wordnegative={}
    for word in wordfreq.keys():
        count=0
        for index,review in enumerate(X):
            if word in review and Y[index]==0:
                count+=1
        wordnegative[word]=count
    return wordnegative

wordnegative=Word_negative(X_train,Y_train,wordfreq)

In [25]:
def total_sentiment(Y ,sentiment):
    length=0
    for i in Y:
        if i == sentiment:
            length+=1
    return length

def conditional_probability(X,Y,word,sentiment,length):
    if sentiment ==1:
        count=0 if word not in wordpositive.keys() else wordpositive[word]
    else:
        count=0 if word not in wordnegative.keys() else wordnegative[word]
    return count/length

def get_sentiment_probability(Y,sentiment):
    count=0
    for i in Y:
        if i==sentiment:
            count+=1
    return count/len(Y)

def calculateBayes(X,Y,word,sentiment,length):
    one=conditional_probability(X,Y,word,sentiment,length)
    two=length/len(Y) #probability of the sentiment
    three=calculate_probability_occurance(X,word)
    count=wordcount[word] if word in wordcount.keys() else 2
    return  (((one*two)+1)/(three+count))

def reviewSentiment(X,Y,review,sentiment):
    sum=1
    length = total_sentiment(Y_train ,1)
    for word in review.split():
        prob = calculateBayes(X,Y,word,sentiment,length)
        sum *= prob
    return sum

def naiveBayes(X,Y,review):
    #check if review sentiment is positive
    positive =reviewSentiment(X,Y,review,1)
    #check if review sentiment is negative
    negative =reviewSentiment(X,Y,review,0)
    if positive > negative:
        return 1
    else :
        return 0
    
def naiveBayes_onWhole(X,Y,x_test):
    ypred=[]
    for review in x_test:
        pred=naiveBayes(X,Y,review)
        ypred.append(pred)
    return ypred

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def calculate_probability_occurance(dataset,word):
    count= 0 if word not in wordcount.keys() else wordcount[word]
    return count/len(dataset)

## calculate accuracy on test Data

In [ ]:
y_pred=naiveBayes_onWhole(X_train,Y_train,X_test)
score=accuracy_metric(Y_test, y_pred)
print('Accuracy on unclean development set : {} %'.format(score))

## Compare with Naive Bayes library from sklearn

In [22]:
X_train=train.iloc[:,0].values
Y_train=train.iloc[:,1].values
for i in range(len(X_train)):
    X_train[i]=getCleanReview(X_train[i])

In [23]:
#set up tfidfvectorizor
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizor=TfidfVectorizer(stop_words='english', max_df=0.7,max_features=10000)

#fit and transform train and test set
tfidf_train=tfidf_vectorizor.fit_transform(X_train).toarray()
tfidf_test=tfidf_vectorizor.transform(X_test).toarray()
tfidf_train.shape

(25000, 10000)

In [24]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
NaiveClassifier = GaussianNB()
NaiveClassifier.fit(tfidf_train, Y_train)

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = NaiveClassifier, X = tfidf_train, y = Y_train, cv = 10)
accuracies.mean()

0.7806399999999999